In [1]:
from getpass import getpass

DASHSCOPE_API_KEY = getpass()

In [2]:
import os

# os.environ['SERPAPI_API_KEY'] = 'a5e0e45bc64feb21f5bcfa9d75dc8bd9fe944a902066ba46b2babf84ed4b6cdd'
# serpapi_api_key = os.environ.get("SERPAPI_API_KEY")

os.environ["DASHSCOPE_API_KEY"] = "sk-7328b2da72c54b178fe9248b3e27c4cf"

In [3]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.messages import HumanMessage

chatLLM = ChatTongyi(
    streaming=True,
)
res = chatLLM.stream([HumanMessage(content="hi")], streaming=True)
for r in res:
    print("chat resp:", r)

chat resp: content='Hello' id='run-29e533c4-1de5-41ad-8fee-ea4d3bf251fb'
chat resp: content='!' id='run-29e533c4-1de5-41ad-8fee-ea4d3bf251fb'
chat resp: content=' How' id='run-29e533c4-1de5-41ad-8fee-ea4d3bf251fb'
chat resp: content=' can I assist you today' id='run-29e533c4-1de5-41ad-8fee-ea4d3bf251fb'
chat resp: content='?' response_metadata={'finish_reason': 'stop', 'request_id': '37ef582f-c5a8-97b4-9091-417450009561', 'token_usage': {'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18}} id='run-29e533c4-1de5-41ad-8fee-ea4d3bf251fb'


In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a helpful assistant that translates English to French."
    ),
    HumanMessage(
        content="Translate this sentence from English to French. I love programming."
    ),
]
chatLLM(messages)

/data/jiangyy/miniconda3/envs/lc/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


AIMessage(content="J'aime programmer.", response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '0401e60b-8241-98f4-9e0f-fecb85777cf6', 'token_usage': {'input_tokens': 36, 'output_tokens': 5, 'total_tokens': 41}}, id='run-8d45c802-2e47-411d-83f7-2fc73ff8a80e-0')

In [26]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


llm = ChatTongyi(model="qwen-turbo")

llm_with_tools = llm.bind_tools([multiply])

print(multiply)

msg = llm_with_tools.invoke("What's 5 times forty two")

print(msg)

name='multiply' description='Multiply two integers together.' args_schema=<class 'pydantic.v1.main.multiplySchema'> func=<function multiply at 0x7fe16cb71cf0>
content='' additional_kwargs={'tool_calls': [{'function': {'name': 'multiply', 'arguments': '{"first_int": 5, "second_int": 42}'}, 'id': '', 'type': 'function'}]} response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'tool_calls', 'request_id': '25737ea9-91d2-9a6a-80c0-dc7c799e051e', 'token_usage': {'input_tokens': 186, 'output_tokens': 25, 'total_tokens': 211}} id='run-4edb945b-61f1-4166-af57-691c945b420d-0' tool_calls=[{'name': 'multiply', 'args': {'first_int': 5, 'second_int': 42}, 'id': ''}]


In [6]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.messages import HumanMessage, SystemMessage

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "当你想知道现在的时间时非常有用。",
            "parameters": {},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "当你想查询指定城市的天气时非常有用。",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "城市或县区，比如北京市、杭州市、余杭区等。",
                    }
                },
            },
            "required": ["location"],
        },
    },
]

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the weather like in San Francisco?"),
]
chatLLM = ChatTongyi()
llm_kwargs = {"tools": tools, "result_format": "message"}
ai_message = chatLLM.bind(**llm_kwargs).invoke(messages)
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'get_current_weather', 'arguments': '{"location": "San Francisco"}'}, 'id': '', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'tool_calls', 'request_id': '601ce8f1-5aeb-9a5e-9686-30576728d6c4', 'token_usage': {'input_tokens': 229, 'output_tokens': 19, 'total_tokens': 248}}, id='run-2bd42aed-8a9a-44b3-a17a-449eda041ff7-0', tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'San Francisco'}, 'id': ''}])

In [25]:
from langchain_community.chat_models import ChatTongyi
from langchain_core.messages import HumanMessage

chatLLM = ChatTongyi(model_name="qwen-vl-max")
image_message = {
    "image": "https://i2.hdslb.com/bfs/archive/374bb291a8840611f987bb4ecd2f427ab03f88cf.jpg",
}
text_message = {
    "text": "introduce the picture",
}
message = HumanMessage(content=[text_message, image_message])
print(message)
answer = chatLLM.invoke([message])
print("--------------------------")
answertxt = answer.to_json()['kwargs']['content'][0]['text']
print(answertxt)

content=[{'text': 'introduce the picture'}, {'image': 'https://i2.hdslb.com/bfs/archive/374bb291a8840611f987bb4ecd2f427ab03f88cf.jpg'}]
--------------------------
This is a beautiful gray long-haired cat with green eyes, lying in the grass and looking forward calmly.
